In [ ]:
from matplotlib.pylab import *
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib.lines import Line2D
import numpy as np
π = pi

In [ ]:
style.use(['dark_background', 'bmh'])
%matplotlib notebook

Car-trailer diagram (inverted image `res/car-trainer-k.png` available as well):
![car-trailer](res/car-trailer-w.png)

Car-trailer equation:
\begin{align}
\dot x &= s \cos \theta_0 \\
\dot y &= s \sin \theta_0 \\
\dot \theta_0 &= \frac{s}{L} \tan \phi \\
\dot \theta_1 &= \frac{s}{d_1} \sin(\theta_1 - \theta_0)
\end{align}
where $s$: signed speed, $\phi$: negative steering angle,

In [ ]:
class Truck:
    def __init__(self, display=False):

        self.W = 1  # car and trailer width, for drawing only
        self.L = 1 * self.W  # car length
        self.d = 4 * self.L  # d_1
        self.s = -0.1  # speed
        self.display = display
        
        self.box = [0, 40, -10, 10]
        if self.display:
            self.f = figure(figsize=(10, 5), num='The truck backer-upper', facecolor='none')
            self.ax = self.f.add_axes([0.01, 0.01, 0.98, 0.98], facecolor='black')
            self.patches = list()
            
            self.ax.axis('equal')
            b = self.box
            self.ax.axis([b[0] - 1, b[1], b[2], b[3]])
            self.ax.set_xticks([]); self.ax.set_yticks([])
            self.ax.axhline(); self.ax.axvline()

        self.reset()

    def randomize_between(self, x_min, x_max, y_min, y_max, θ0_min, θ0_max, θ1_min, θ1_max):
        #convert to radians
        θ0_min = θ0_min * π/ 180
        θ0_max = θ0_max * π/ 180
        θ1_min = θ1_min * π/ 180
        θ1_max = θ1_max * π/ 180

        #randomize values between min and max
        self.θ0 = np.random.uniform(θ0_min, θ0_max) #(random() - (θ0_max + θ0_min)/2) * (θ0_max - θ0_min)
        self.θ1 = np.random.uniform(θ1_min, θ1_max) + self.θ0 #(random() - (θ1_max + θ1_min)/2) * (θ1_max - θ1_min)
        self.x = np.random.uniform(x_min, x_max) #(random() - (x_max + x_min)/2) * (x_max - x_min)
        self.y = np.random.uniform(y_min, y_max) #(random() - (y_max + y_min)/2) * (y_max - y_min)
        
        # If poorly initialise, then re-initialise
        if not self.valid():
            self.randomize_between(x_min, x_max, y_min, y_max, θ0_min, θ0_max, θ1_min, θ1_max)
        
        # Draw, if display is True
        if self.display: self.draw()

    
    def reset(self, ϕ=0):
        self.ϕ = ϕ  # car initial steering angle

        self.θ0 = random() * 2 * π  # 0 <= ϑ₀ < 2π
        self.θ1 = (random() - 0.5) * π / 2 + self.θ0  # -π/4 <= ϑ₁ - ϑ₀ < π/4
        self.x = (random() * .75 + 0.25) * self.box[1]
        self.y = (random() - 0.5) * (self.box[3] - self.box[2])
    
        # self.θ0 = deg2rad(30)  # car initial direction
        # self.θ1 = deg2rad(-30)  # trailer initial direction
        # self.x, self.y = 20, -5  # initial car coordinates
        
        self.θ0 = random() * 2 * π  # 0 <= ϑ₀ < 2π
        self.θ1 = (random() - 0.5) * π / 2 + self.θ0  # -π/4 <= ϑ₁ - ϑ₀ < π/4
        self.x = (random() * .75 + 0.25) * self.box[1]
        self.y = (random() - 0.5) * (self.box[3] - self.box[2])
        
        # If poorly initialise, then re-initialise
        if not self.valid():
            self.reset(ϕ)
        
        # Draw, if display is True
        if self.display: self.draw()
    
    def step(self, ϕ=0, dt=1):
        
        # Check for illegal conditions
        if self.is_jackknifed():
            # print('The truck is jackknifed!')
            return (self.x, self.y, self.θ0, *self._traler_xy(), self.θ1)
        
        if self.is_offscreen():
            # print('The car or trailer is off screen')
            return (self.x, self.y, self.θ0, *self._traler_xy(), self.θ1)
        
        self.ϕ = ϕ
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        
        # Perform state update
        self.x += s * cos(θ0) * dt
        self.y += s * sin(θ0) * dt
        self.θ0 += s / L * tan(ϕ) * dt
        self.θ1 += s / d * sin(θ0 - θ1) * dt
        
        return (self.x, self.y, self.θ0, *self._traler_xy(), self.θ1)
    
    def state(self):
        return (self.x, self.y, self.θ0, *self._traler_xy(), self.θ1)
    
    def _get_atributes(self):
        return (
            self.x, self.y, self.W, self.L, self.d, self.s,
            self.θ0, self.θ1, self.ϕ
        )
    
    def _traler_xy(self):
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        return x - d * cos(θ1), y - d * sin(θ1)
        
    def is_jackknifed(self):
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        return abs(θ0 - θ1) * 180 / π > 90
    
    def is_offscreen(self):
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        
        x1, y1 = x + 1.5 * L * cos(θ0), y + 1.5 * L * sin(θ0)
        x2, y2 = self._traler_xy()
        
        b = self.box
        return not (
            b[0] <= x1 <= b[1] and b[2] <= y1 <= b[3] and
            b[0] <= x2 <= b[1] and b[2] <= y2 <= b[3]
        )
    
    def is_atdock(self):
        x,y = self._traler_xy()
        return (x<0.1) and (abs(y)<0.1)

    def can_continue(self):
        return not self.is_atdock() or  not self.is_offscreen()
        
    def valid(self):
        return not self.is_jackknifed() and not self.is_offscreen()
        
    def draw(self):
        if not self.display: return
        if self.patches: self.clear()
        self._draw_car()
        self._draw_trailer()
        self.f.canvas.draw()
            
    def clear(self):
        for p in self.patches:
            p.remove()
        self.patches = list()
        
    def _draw_car(self):
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        ax = self.ax
        
        x1, y1 = x + L / 2 * cos(θ0), y + L / 2 * sin(θ0)
        bar = Line2D((x, x1), (y, y1), lw=5, color='C2', alpha=0.8)
        ax.add_line(bar)

        car = Rectangle(
            (x1, y1 - W / 2), L, W, 0, color='C2', alpha=0.8, transform=
            matplotlib.transforms.Affine2D().rotate_deg_around(x1, y1, θ0 * 180 / π) +
            ax.transData
        )
        ax.add_patch(car)

        x2, y2 = x1 + L / 2 ** 0.5 * cos(θ0 + π / 4), y1 + L / 2 ** 0.5 * sin(θ0 + π / 4)
        left_wheel = Line2D(
            (x2 - L / 4 * cos(θ0 + ϕ), x2 + L / 4 * cos(θ0 + ϕ)),
            (y2 - L / 4 * sin(θ0 + ϕ), y2 + L / 4 * sin(θ0 + ϕ)),
            lw=3, color='C5', alpha=1)
        ax.add_line(left_wheel)

        x3, y3 = x1 + L / 2 ** 0.5 * cos(π / 4 - θ0), y1 - L / 2 ** 0.5 * sin(π / 4 - θ0)
        right_wheel = Line2D(
            (x3 - L / 4 * cos(θ0 + ϕ), x3 + L / 4 * cos(θ0 + ϕ)),
            (y3 - L / 4 * sin(θ0 + ϕ), y3 + L / 4 * sin(θ0 + ϕ)),
            lw=3, color='C5', alpha=1)
        ax.add_line(right_wheel)
        
        self.patches += [car, bar, left_wheel, right_wheel]
        
    def _draw_trailer(self):
        x, y, W, L, d, s, θ0, θ1, ϕ = self._get_atributes()
        ax = self.ax
            
        x, y = x - d * cos(θ1), y - d * sin(θ1) - W / 2
        trailer = Rectangle(
            (x, y), d, W, 0, color='C0', alpha=0.8, transform=
            matplotlib.transforms.Affine2D().rotate_deg_around(x, y + W/2, θ1 * 180 / π) +
            ax.transData
        )
        ax.add_patch(trailer)
        
        self.patches += [trailer]

In [ ]:
truck = Truck(display=True)

In [ ]:
ϕ = deg2rad(-35)  # positive left, negative right
truck.step(ϕ)
truck.draw()

In [ ]:
truck.reset()

In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD
from tqdm import tqdm

In [ ]:
# Build expert data set

episodes = 10
inputs = list()
outputs = list()
truck = Truck(); episodes = 10_000  # uncooment for creating the data set

for episode in tqdm(range(episodes)):
    
    truck.reset()
    
    while truck.valid():
        initial_state = truck.state()
        ϕ = (random() - 0.5) * π / 2
        inputs.append((ϕ, *initial_state))
        outputs.append(truck.step(ϕ))
        truck.draw()

In [ ]:
len(inputs), len(outputs)

In [ ]:
state_size = 6
steering_size = 1
hidden_units_e = 45

emulator = nn.Sequential(
    nn.Linear(steering_size + state_size, hidden_units_e),
    nn.ReLU(),
    nn.Linear(hidden_units_e, state_size)
)

optimiser_e = SGD(emulator.parameters(), lr=0.005)
criterion = nn.MSELoss()

In [ ]:
tensor_inputs = torch.Tensor(inputs)
tensor_outputs = torch.Tensor(outputs)

In [ ]:
mean = tensor_inputs.mean(0)
std = tensor_inputs.std(0)
tensor_inputs = (tensor_inputs - mean) / std
tensor_outputs = (tensor_outputs - mean[1:]) / std[1:]

In [ ]:
# Split the data into 80:20 for test:train.
test_size = int(len(tensor_inputs) * 0.8)
print(len(tensor_inputs), test_size)

train_inputs = tensor_inputs[:test_size]
train_outputs = tensor_outputs[:test_size]
test_inputs = tensor_inputs[test_size:]
test_outputs = tensor_outputs[test_size:]

In [ ]:
len(train_inputs)

In [ ]:
# Emulator training
cnt = 0
for i in torch.randperm(len(train_inputs)):
    ϕ_state = train_inputs[i]
    next_state_prediction = emulator(ϕ_state)
    
    next_state = train_outputs[i]
    loss = criterion(next_state_prediction, next_state)
    
    optimiser_e.zero_grad()
    loss.backward()
    optimiser_e.step()
    
    if cnt == 0 or (cnt + 1) % 1000 == 0:
        print(f'{cnt + 1:4d} / {len(train_inputs)}, {loss.item():.10f}')
    cnt += 1

In [ ]:
# Test
total_loss = 0
with torch.no_grad():
    for idx, ϕ_state in enumerate(test_inputs):
        next_state_prediction = emulator(ϕ_state)

        next_state = test_outputs[idx]
        total_loss += criterion(next_state_prediction, next_state).item()

ave_test_loss = total_loss/test_size
print(f'Test loss: {ave_test_loss:.10f}')

In [ ]:

# Here you need to insert the code for training the controller
# by using the emulator for backpropagation
import numpy as np
import torch.optim as optim

class c_model(torch.nn.Module):
    def __init__(self):
        super(c_model, self).__init__()

        self.layers = nn.Sequential(
                        nn.Linear(state_size, 45),
                        nn.ReLU(),
                        nn.Linear(45, 45),
                        nn.ReLU(),
                        nn.Linear(45, 1),
                        nn.Tanh(),
                    )

    def forward(self, x):

        n_steer = self.layers(x)
        
        return torch.mul(n_steer, (π/2))

Controller = c_model()

#Change this to start at a different lesson
#ONLY DO THIS IF YOU HAVE TRAINED AT LEAST 2 LESSONS AND THE FILE IS SAVED
starting_lesson = 1

if starting_lesson > 1:
    checkpoint = torch.load("controller_weights_" + str(starting_lesson))
    Controller.load_state_dict(checkpoint)

#initialize optimizer
optimiser_c = SGD(Controller.parameters(), lr=0.0001, momentum=0.9)
criterion = nn.MSELoss()

#parameters for training lessons
#CHANGE THESE TO SUITE YOUR NEEDS
num_lessons = 20
num_samples =1_000
max_steps = 1_000

#define lessons
x_min = torch.linspace(4, 30, num_lessons)
x_max = torch.linspace(6, 35, num_lessons)

y_min = torch.linspace(-1, -9, num_lessons)
y_max = torch.linspace(1, 9, num_lessons)

θ0_min = torch.linspace(0, -90, num_lessons)
θ0_max = torch.linspace(0, 90, num_lessons)

θ1offset_min = torch.linspace(0, -45, num_lessons)
θ1offset_max = torch.linspace(0, 45, num_lessons)

truck_lessons = []

criterion = torch.nn.MSELoss()

#populate array of
for i in range(starting_lesson-1, num_lessons):
    truck_lessons.append([(x_min[i], x_max[i]), (y_min[i], y_max[i]), (θ0_min[i], θ0_max[i]), (θ1offset_min[i], θ1offset_max[i])])


# If you succeed, feel free to send a PR

#freeze emulator weights
emulator
emulator.requires_grad_(False)

l = starting_lesson - 1
#start training from easy to hard lessons
for lesson in truck_lessons:
    print("Start Lesson " + str(l+1))
    cnt = 0

    #start training each sample
    for s in range(num_samples):

        truck.randomize_between(lesson[0][0], lesson[0][1], lesson[1][0], lesson[1][1], lesson[2][0], lesson[2][1], lesson[3][0], lesson[3][1])
        canContinue = True
        no_error = True
         
        i=0

        state = torch.tensor(truck.state(), requires_grad=True, dtype=torch.float32)

        states = [state]
        while canContinue:
            # state = torch.Tensor(truck.state())
            ϕ = Controller(state)
            # e_input = torch.cat((ϕ,state))
            new = truck.step(ϕ.item())
            state = torch.tensor(new, requires_grad=True, dtype=torch.float32)
            states.append(state)
            canContinue = truck.can_continue()
            # state = emulator(e_input)
            if(canContinue and i >= max_steps):
                no_error = False
                break

            i += 1

        if i == 0:
            no_error = False
        
        if no_error:
            optimiser_c.zero_grad()
            states.reverse()
            last_state = states.pop(0)
            dock = torch.tensor([0,0.0, 0.0,0])
            loss = criterion(last_state[2:], dock)
            loss.backward()
            grad = last_state.grad
            
            for state in states:
                ϕ = Controller(state)
                e_input = torch.cat((ϕ,state))
                out = emulator(e_input)
                out.backward(grad)
                grad = state.grad

            optimiser_c.step()

            cnt += 1
            print(f' lesson: {l + 1:4d} / {num_lessons}, sample: {s+1 :4d} / {num_samples}, {loss.item():.10f}')
              
    print("Saving weights for lesson " + str(l+1))
    torch.save(Controller.state_dict(), "controller_weights_" + str(l+1))     
    l += 1
    

In [ ]:
#Test Controller
truck.reset()

while(True):
    state = torch.Tensor(truck.state())
    ϕ = Controller(state)
    truck.step(ϕ.item())
    truck.draw()
    